In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.table import Table, hstack, vstack
from sklearn.neighbors import KernelDensity
from tqdm import tqdm
import astropy.units as u
from astropy.coordinates import SkyCoord
import mpl_scatter_density
import corner

import numpy as np
from matplotlib import cm
from matplotlib.patches import Rectangle
from matplotlib.colors import ListedColormap


In [ ]:
params = {
    "legend.fontsize": "x-large",
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
    "figure.facecolor": "w",
    "xtick.top": True,
    "ytick.right": True,
    "xtick.direction": "in",
    "ytick.direction": "in",
    "font.family": "serif",
    "mathtext.fontset": "dejavuserif"
}
plt.rcParams.update(params)

def cmap_white(cmap_name):
    """Returns a colormap with white as the lowest value color."""
    import numpy as np
    try:
        from matplotlib import cm
        from matplotlib.colors import ListedColormap
        cmap = cm.get_cmap(cmap_name, 256)
    except ValueError:
        import seaborn as sns
        cmap = sns.color_palette("flare", as_cmap=True)
    newcolors = cmap(np.linspace(0, 1, 256))
    white = np.array([1, 1, 1, 0])
    newcolors[:1, :] = white
    cmap_white = ListedColormap(newcolors)
    return cmap_white

### SQL Query for HSC PDR3

```sql

SELECT
    main.object_id,
    main.ra,
    main.dec,

    ------- flux and flux errors -------
    main2.g_psfflux_flux, main2.r_psfflux_flux, main2.i_psfflux_flux, main2.z_psfflux_flux, main2.y_psfflux_flux,
    main2.g_psfflux_fluxerr, main2.r_psfflux_fluxerr, main2.i_psfflux_fluxerr, main2.z_psfflux_fluxerr, main2.y_psfflux_fluxerr,
    main.g_cmodel_flux, main.r_cmodel_flux, main.i_cmodel_flux, main.z_cmodel_flux, main.y_cmodel_flux,
    main.g_cmodel_fluxerr, main.r_cmodel_fluxerr, main.i_cmodel_fluxerr, main.z_cmodel_fluxerr, main.y_cmodel_fluxerr,
------- Fiber flux and flux errors -------
main4.g_convolvedflux_2_15_flux as g_fiber_flux, main4.r_convolvedflux_2_15_flux as r_fiber_flux,
main4.i_convolvedflux_2_15_flux as i_fiber_flux, main4.z_convolvedflux_2_15_flux as z_fiber_flux, main4.y_convolvedflux_2_15_flux as y_fiber_flux,
main4.g_convolvedflux_2_15_fluxerr as g_fiber_fluxerr, main4.r_convolvedflux_2_15_fluxerr as r_fiber_fluxerr, main4.i_convolvedflux_2_15_fluxerr as i_fiber_fluxerr,
main4.z_convolvedflux_2_15_fluxerr as z_fiber_fluxerr, main4.y_convolvedflux_2_15_fluxerr as y_fiber_fluxerr,

main5.g_undeblended_convolvedflux_2_15_flux as g_fiber_tot_flux, main5.r_undeblended_convolvedflux_2_15_flux as r_fiber_tot_flux,
main5.i_undeblended_convolvedflux_2_15_flux as i_fiber_tot_flux, main5.z_undeblended_convolvedflux_2_15_flux as z_fiber_tot_flux, main5.y_undeblended_convolvedflux_2_15_flux as y_fiber_tot_flux,
main5.g_undeblended_convolvedflux_2_15_fluxerr as g_fiber_tot_fluxerr, main5.r_undeblended_convolvedflux_2_15_fluxerr as r_fiber_tot_fluxerr, main5.i_undeblended_convolvedflux_2_15_fluxerr as i_fiber_tot_fluxerr,
main5.z_undeblended_convolvedflux_2_15_fluxerr as z_fiber_tot_fluxerr, main5.y_undeblended_convolvedflux_2_15_fluxerr as y_fiber_tot_fluxerr,
    ------- extinction -------
    main.a_g, main.a_r, main.a_i, main.a_z, main.a_y,

    ------- fraction of flux in de Vaucouleur component -------
    main.g_cmodel_fracdev, main.r_cmodel_fracdev, main.i_cmodel_fracdev, main.z_cmodel_fracdev,
     ------- shape measurements -------
    main.g_extendedness_value, main.r_extendedness_value, main.i_extendedness_value, main.z_extendedness_value,
    main.g_extendedness_flag, main.r_extendedness_flag, main.i_extendedness_flag, main.z_extendedness_flag,
    main2.i_sdssshape_shape11, main2.i_sdssshape_shape22, main2.i_sdssshape_shape12,
    main2.i_sdssshape_shape11err, main2.i_sdssshape_shape22err, main2.i_sdssshape_shape12err,
    ------- flags -------
    main2.g_sdsscentroid_flag, main2.r_sdsscentroid_flag, main2.i_sdsscentroid_flag, main2.z_sdsscentroid_flag, main2.y_sdsscentroid_flag, 
    main.g_pixelflags_edge, main.r_pixelflags_edge, main.i_pixelflags_edge, main.z_pixelflags_edge, main.y_pixelflags_edge, 
    main.g_pixelflags_interpolatedcenter, main.r_pixelflags_interpolatedcenter, main.i_pixelflags_interpolatedcenter, main.z_pixelflags_interpolatedcenter, main.y_pixelflags_interpolatedcenter, 
    main.g_pixelflags_saturatedcenter, main.r_pixelflags_saturatedcenter, main.i_pixelflags_saturatedcenter, main.z_pixelflags_saturatedcenter, main.y_pixelflags_saturatedcenter, 
    main.g_pixelflags_crcenter, main.r_pixelflags_crcenter, main.i_pixelflags_crcenter, main.z_pixelflags_crcenter, main.y_pixelflags_crcenter, 
    main.g_pixelflags_bad, main.r_pixelflags_bad, main.i_pixelflags_bad, main.z_pixelflags_bad, main.y_pixelflags_bad, 
    main.g_cmodel_flag, main.r_cmodel_flag, main.i_cmodel_flag, main.z_cmodel_flag, main.y_cmodel_flag

FROM
    pdr3_wide.forced main
    LEFT JOIN pdr3_wide.forced2 main2 USING (object_id)
    LEFT JOIN pdr3_wide.forced4 main4 USING (object_id)
    LEFT JOIN pdr3_wide.forced5 main5 USING (object_id)

WHERE
    isprimary
    AND i_cmodel_mag<24.8
    AND boxSearch(coord, 33.5, 37.5, -7.0, -3.0) ---XMM
    ---AND boxSearch(coord, 148, 152, 4, 0) ---COSMOS
```


Redshift Query:

```sql
SELECT
    main.object_id,
    main.specz_name,
    main.specz_redshift,
    main.specz_redshift,
    main.specz_redshift_err,
    main.specz_flag_homogeneous,
    main.d_pos,
    main.d_mag,
    main.specz_duplicationflag,
    main.specz_zerrflag


  
FROM pdr3_wide.forced
RIGHT JOIN pdr3_wide.specz main USING (object_id)
    
    

WHERE
    isprimary
    AND i_cmodel_mag<24.4
    AND boxSearch(coord, 33.5, 37.5, -7.0, -3.0)
```

In [ ]:
base_path = Path("/global/cfs/cdirs/desi/users/bid13/DESI_II/")
# hsc_path = base_path / "HSC_XMM_I_mag_lim_24.4_new.fits"
patch = "COSMOS"
hsc_path = base_path / f"HSC_{patch}_I_mag_lim_24.8.fits"

# redshift_path = base_path / f"Redshifts_HSC_{patch}_I_mag_lim_24.4.fits"
# redmapper_path = base_path / "des_redmapper_0.40_fainter.fits"
# hsc_path = Path("/global/cfs/cdirs/desi/target/analysis/truth/parent/hsc-pdr3-dud-rev-no_mag_limit-reduced.fits") #revised version
# hsc_path = Path("/global/cfs/cdirs/desi/target/analysis/truth/parent/hsc-pdr3-dud-no_mag_limit-reduced.fits") # with VAC

In [ ]:
def flux_to_mag(flux):
    return -2.5*np.log10(flux*1e-9) + 8.90

In [ ]:
hsc_cat = Table.read(hsc_path).to_pandas()
# extinction corrected mags (extinction is negligible for XMM-LSS)
hsc_cat["i_mag"] = flux_to_mag(hsc_cat["i_cmodel_flux"])-hsc_cat["a_i"]
hsc_cat["r_mag"] = flux_to_mag(hsc_cat["r_cmodel_flux"])-hsc_cat["a_r"]
hsc_cat["z_mag"] = flux_to_mag(hsc_cat["z_cmodel_flux"])-hsc_cat["a_z"]
hsc_cat["g_mag"] = flux_to_mag(hsc_cat["g_cmodel_flux"])-hsc_cat["a_g"]

hsc_cat["g_fiber_mag"] = flux_to_mag(hsc_cat["g_fiber_flux"])-hsc_cat["a_g"]
hsc_cat["i_fiber_mag"] = flux_to_mag(hsc_cat["i_fiber_flux"])-hsc_cat["a_i"]
hsc_cat["r_fiber_mag"] = flux_to_mag(hsc_cat["r_fiber_flux"])-hsc_cat["a_r"]
hsc_cat["z_fiber_mag"] = flux_to_mag(hsc_cat["z_fiber_flux"])-hsc_cat["a_z"]
# hsc_cat["i_fiber_tot_mag"] = flux_to_mag(hsc_cat["i_fiber_tot_flux"])-hsc_cat["a_i"]

In [ ]:
# redshift_cat = Table.read(redshift_path).to_pandas()
# # All are unique in this one
# redshift_cat = redshift_cat[redshift_cat["specz_flag_homogeneous"]]
# hsc_cat = pd.merge(hsc_cat,redshift_cat,how="left", on="object_id")

In [ ]:
## Quality cuts
# valid I-band flux
qmask = np.isfinite(hsc_cat["i_cmodel_flux"]) & (hsc_cat["i_cmodel_flux"]>0)
#cmodel fit not failed
qmask &= (~hsc_cat["i_cmodel_flag"].values)
#General Failure Flag
qmask &= (~hsc_cat["i_sdsscentroid_flag"].values)

# Possible cuts: Bright objects nearby, bad pixels

#star-galaxy separation (is point source in I band)
# mask &= (hsc_cat["i_extendedness_value"]>0)


i_min = 22
i_max = 24.5
mask = (hsc_cat["i_mag"] <i_max) & (hsc_cat["i_mag"] >i_min)
i_fiber_min = 22
i_fiber_max = 24.75
mask &= (hsc_cat["i_fiber_mag"] <i_fiber_max) & (hsc_cat["i_fiber_mag"] >i_fiber_min)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.scatter(hsc_cat["i_mag"], hsc_cat["i_fiber_mag"],s=1, alpha=0.01)
ax.set_aspect('equal')
x = np.linspace(15,25,100)
ax.plot(x,x, ls="--", c="k", alpha=0.5)
ax.set_xlim(18,25)
ax.set_ylim(18,25)
ax.set_xlabel("$i$ mag")
ax.set_ylabel("$i$ fiber mag")

In [ ]:
sels_cat = hsc_cat[qmask & mask]
sels_cat = sels_cat.reset_index()

In [ ]:
sels_cat["i_mag"].min()

In [ ]:
plt.hist(sels_cat["i_fiber_mag"], bins=100, range=(21,30), histtype="step")
plt.xlabel("i fiber mag")
plt.axvline(24.5)
plt.yscale("log")
plt.xlim(21,30)
plt.show()

In [ ]:
plt.hist(hsc_cat["i_mag"],bins=50, histtype="step", label="LSST Y1")
plt.hist(hsc_cat["i_mag"][hsc_cat["z_mag"]<20.8],bins=50, histtype="step", label = "~DC3R2")
plt.hist(hsc_cat["i_mag"][hsc_cat["r_mag"]<20.1],bins=50, histtype="step", label = "~BGS Faint")
plt.hist(hsc_cat["i_mag"][hsc_cat["g_mag"]<23.4],bins=50, histtype="step", label = "~ELG")
plt.hist(hsc_cat["i_mag"][hsc_cat["z_mag"]<21],bins=50, histtype="step", label = "~LRG")
plt.axvline(21,c="k",ls="--")
plt.text(21.1,2,'21',fontsize=15)

plt.axvline(22.5,c="k",ls="--")
plt.text(22.6,2,'22.5',fontsize=15)

plt.axvline(19.5,c="k",ls="--")
plt.text(19.6,2,'19.5',fontsize=15)

plt.xlabel("$i$ mag", fontsize=20)
plt.yscale("log")
plt.legend(loc="upper left")

In [ ]:
hist, bin_edges = np.histogram(sels_cat["i_mag"], bins=10000,density=True)
bin_edges[-1] = i_max
bin_edges[0] = i_min

In [ ]:
bin_membership = np.digitize(sels_cat["i_mag"], bin_edges)

In [ ]:
prob = hist[bin_membership-1]

weights = 1/prob
weights = weights/weights.sum()

In [ ]:
rng = np.random.default_rng(seed=42)
resample_idx = rng.choice(np.arange(len(sels_cat["i_mag"])), size=100000, replace=False, p=weights)

In [ ]:
final_sel = sels_cat.iloc[resample_idx]

In [ ]:
plt.figure(figsize=(7,5))
plt.hist(sels_cat["i_mag"],bins=100, histtype="step", density=True, label="Magnitude Limited Sample")
plt.hist(final_sel["i_mag"],bins=100, histtype="step", density=True, label="Uniform Magnitude Sample")
# plt.hist(sels_cat["i_mag"][resample_idx][np.isfinite(sels_cat["specz_redshift"][resample_idx])],bins=50, histtype="step", density=True, label="Known Redshifts")
plt.axhline(np.mean(np.histogram(sels_cat["i_mag"][resample_idx],bins=100, density=True)[0]), c="k", ls="--")
plt.xlabel("$i$-band Magnitude", fontsize=20)
plt.ylabel("Normalized Frequency", fontsize=20)
# plt.yscale("log")
plt.legend()
plt.savefig("LSSTY1_i_distribution.pdf", bbox_inches="tight")

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1, 1, 1, projection='scatter_density')
ax.scatter_density(final_sel["i_mag"].values, final_sel["i_fiber_mag"].values, label="HSC-WL", 
                      cmap=cmap_white("viridis"),vmin=0,vmax=8) #rocket
x = np.linspace(22,24.5,100)
ax.set_aspect("equal")
ax.set_xlabel("$i$-mag",fontsize=20)
ax.set_ylabel("$i$-fiber-mag",fontsize=20)
plt.savefig("LSST_Y1_mag_box.pdf", bbox_inches="tight")

In [ ]:
# small_cat = hsc_cat.sample(n=10000)
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.scatter(sels_cat.loc[resample_idx, "ra"],sels_cat.loc[resample_idx, "dec"],marker=".",s=1, alpha=1)
# plt.scatter(small_cat["ra"],small_cat["dec"],marker=".",s=1)
# plt.scatter(redm["RA"],redm["DEC"],marker=".",s=5, c="r")
ra_min = 148
ra_max = 152
dec_min = 0
dec_max = 4

ax.set_xlabel("RA $\degree$", fontsize=20)
ax.set_ylabel("DEC $\degree$", fontsize=20)
ax.set_xticks(np.arange(ra_min, ra_max+1, 1))
ax.set_xticks(np.arange(ra_min, ra_max+1, 0.5), minor=True)
ax.set_yticks(np.arange(dec_min, dec_max+1, 1))
ax.set_yticks(np.arange(dec_min, dec_max+1, 0.5), minor=True)
ax.set_xlim(ra_max+0.2, ra_min-0.2)
ax.set_ylim(dec_min-0.2, dec_max+0.2)
# And a corresponding grid
ax.grid(which='both')

# Or if you want different settings for the grids:
ax.grid(which='minor', alpha=0.5)
ax.grid(which='major', alpha=0.8)
plt.savefig("cosmos_patch.png", dpi=300)

In [ ]:
blank = np.zeros(len(final_sel))
Table({"RA":final_sel["ra"], "DEC":final_sel["dec"], "PMRA":blank, "PMDEC":blank,
       'REF_EPOCH':blank+2000, "OVERRIDE":(blank+1).astype(bool),"I_MAG": final_sel["i_mag"]}, ).write(base_path / f"{patch}_LSSTY1_target_list.fits", overwrite=True)

In [ ]:
# # from joblib import Parallel, delayed

# # def intersect_score(i):
# #     rng = np.random.default_rng(seed=i)
# #     resample_idx = rng.choice(np.arange(len(sels_cat["i_mag"])), size=10000, replace=False, p=weights)
# #     return np.isfinite(sels_cat["specz_redshift"][resample_idx]).sum()

# # score = Parallel(n_jobs=16)(delayed(intersect_score)(i) for i in tqdm(range(1000)))


# score = []
# for i in tqdm(range(10000)):
#     rng = np.random.default_rng(seed=i)
#     resample_idx = rng.choice(np.arange(len(sels_cat["i_mag"])), size=50000, replace=False, p=weights)
#     score.append(np.isfinite(sels_cat["specz_redshift"][resample_idx]).sum())
# score=np.array(score)

# Other targets

### HSC WL

In [ ]:
# data_path = base_path / "s16a_weak_lensing_medium.hdf5"
# wl_cat = pd.read_hdf(data_path, key="source")

# wl_coord = SkyCoord(ra=wl_cat["ra"].values*u.degree, dec=wl_cat["dec"].values*u.degree)
# hsc_coord = SkyCoord(ra=hsc_cat["ra"].values*u.degree, dec=hsc_cat["dec"].values*u.degree)

# idx, d2d, d3d = hsc_coord.match_to_catalog_sky(wl_coord)


# idx_mask = d2d.arcsec<1


# match_cat = hsc_cat[idx_mask]


In [ ]:
# match_cat.to_hdf(base_path / "hsc_WL_XMM", key="cat")

In [ ]:
hsc_wl = pd.read_hdf(base_path / "hsc_WL_XMM.hdf5")

### Merian

In [ ]:
hsc_coord = SkyCoord(ra=hsc_cat["ra"].values*u.degree, dec=hsc_cat["dec"].values*u.degree)

m1_cat = Table.read(base_path / "DESI_proposal_MS1.fits")
m1_coord = SkyCoord(ra=m1_cat["ALPHA_J2000"], dec=m1_cat["DELTA_J2000"])
idx, d2d, d3d = hsc_coord.match_to_catalog_sky(m1_coord)
idx_mask = d2d.arcsec<1
m1_match_cat = hsc_cat[idx_mask]
# print(len(m1_match_cat)/len(m1_cat))
print(len(m1_match_cat)/len(final_sel))

m2_cat = Table.read(base_path / "DESI_proposal_MS2.fits")
m2_coord = SkyCoord(ra=m2_cat["ALPHA_J2000"], dec=m2_cat["DELTA_J2000"])
idx, d2d, d3d = hsc_coord.match_to_catalog_sky(m2_coord)
idx_mask = d2d.arcsec<1
m2_match_cat = hsc_cat[idx_mask]
# print(len(m2_match_cat)/len(m2_cat))
print(len(m2_match_cat)/len(final_sel))

m3_cat = Table.read(base_path / "MS3_objects.fits")
m3_coord = SkyCoord(ra=m3_cat["ALPHA_J2000"]*u.degree, dec=m3_cat["DELTA_J2000"]*u.degree)
idx, d2d, d3d = hsc_coord.match_to_catalog_sky(m3_coord)
idx_mask = d2d.arcsec<1
m3_match_cat = hsc_cat[idx_mask]
# print(len(m3_match_cat)/len(m3_cat))
print(len(m3_match_cat)/len(final_sel))

### LOWZ

In [ ]:
lowz_cat = Table.read(base_path / "LOWZ_COSMOS_alltiers.fits")
lowz_coord = SkyCoord(ra=lowz_cat["RA"]*u.degree,dec=lowz_cat["DEC"]*u.degree)

idx, d2d, d3d = hsc_coord.match_to_catalog_sky(lowz_coord)
idx_mask = d2d.arcsec<0.75
lowz_match_cat = hsc_cat[idx_mask]
# print(len(lowz_match_cat)/len(lowz_cat))
print(len(lowz_match_cat)/len(final_sel))

### Summary plot

In [ ]:
fig, axs = plt.subplots(1,3, subplot_kw={"projection":'scatter_density'}, figsize=(24,8))
axs=np.ravel(axs)

axs[0].scatter_density(lowz_match_cat["i_mag"], lowz_match_cat["i_fiber_mag"],cmap=cmap_white("rocket"), vmin=0, vmax=20)
axs[0].set_title("LOWZ", fontsize=25)


axs[1].scatter(m1_match_cat["i_mag"], m1_match_cat["i_fiber_mag"], label="MS1", marker=".", s=4, c="k")
axs[1].scatter(m2_match_cat["i_mag"], m2_match_cat["i_fiber_mag"], label="MS2", marker=".", s=4,c=(0.8985026 , 0.22463251, 0.25108745, 1.)) 
axs[1].set_title("MERIAN", fontsize=25)
lgnd = axs[1].legend()
lgnd.legendHandles[0]._sizes = [50]
lgnd.legendHandles[1]._sizes = [50]

pmask = np.isfinite(hsc_wl["i_mag"].values) & np.isfinite(hsc_wl["i_fiber_mag"].values)
axs[2].scatter_density(hsc_wl["i_mag"][pmask].values, hsc_wl["i_fiber_mag"][pmask].values, label="HSC-WL", 
                      cmap=cmap_white("rocket"),vmin=0,vmax=50, dpi=80)
axs[2].set_title("HSC-WL", fontsize=25)


axs[2].scatter_density(final_sel["i_mag"].values, final_sel["i_fiber_mag"].values, label="HSC-WL", 
                      cmap=cmap_white("gray"),vmin=0,vmax=25)
axs[2].add_patch(Rectangle((22, 22), 2.5, 2.75, facecolor="none", edgecolor="k",lw=2))
axs[2].text(22.6,22.2, "LSST-Y1", fontsize=20)

# corner.hist2d(final_sel["i_mag"].values, final_sel["i_fiber_mag"].values,
#               ax=axs[2], no_fill_contours=False, plot_density=True,color="red")
max_mag = 18
min_mag = 25.5
x = np.linspace(max_mag,min_mag,100)
for ax in axs:
    ax.plot(x,x, ls="--", c="k", alpha=0.5)
    ax.set_ylabel("$i$-fiber-mag", fontsize=20)
    ax.set_xlabel("$i$-mag", fontsize=20)
    ax.set_aspect('equal')
    ax.set_ylim(max_mag,min_mag)
    ax.set_xlim(max_mag,min_mag)
plt.savefig("ALL_i_distribution_2D.pdf", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(16,6), sharey=True)
ax[0].hist(lowz_match_cat["i_mag"], histtype="step", bins=50, density=True, label="LOWZ",lw=2)
ax[0].hist(m1_match_cat["i_mag"], histtype="step", bins=50, density=True, label="MS1",lw=2)
ax[0].hist(m2_match_cat["i_mag"], histtype="step", bins=50, density=True, label="MS2",lw=2)
ax[0].hist(final_sel["i_mag"], histtype="step", bins=50, density=True, label="LSSTY1",lw=2)
ax[0].set_xlim(17,25)
ax[0].set_yscale("log")
ax[0].set_xlabel("$i$-mag", fontsize=25)
ax[0].legend()

ax[0].set_ylabel("Normalized Density", fontsize=25)


ax[1].hist(lowz_match_cat["i_fiber_mag"], histtype="step", bins=50,density=True,lw=2)
ax[1].hist(m1_match_cat["i_fiber_mag"], histtype="step", bins=50, density=True, label="MS1",lw=2)
ax[1].hist(m2_match_cat["i_fiber_mag"], histtype="step", bins=50, density=True, label="MS2",lw=2)

ax[1].hist(final_sel["i_fiber_mag"], histtype="step", bins=50, density=True, label="LSSTY1",lw=2)
ax[1].set_xlim(18,27)
ax[1].set_yscale("log")
ax[1].set_xlabel("$i$-fiber-mag", fontsize=25)
plt.savefig("ALL_i_distribution_1D.pdf", bbox_inches="tight")

### Explore the fiberassign files

In [ ]:
fa_path = Path("/global/cfs/cdirs/desi/survey/fiberassign/special/tertiary/0015/tertiary-targets-0015-assign.fits")

In [ ]:
fa = Table.read(fa_path, hdu=1)
names = [name for name in fa.colnames if len(fa[name].shape) <= 1]
fa = fa[names].to_pandas()

In [ ]:
fa = fa[fa["TERTIARY_TARGET"].isin([b"LSST_WLY1_HIP", b"LSST_WLY1_LOP"])]
fa = fa[fa["NASSIGN"]>0]

In [ ]:
plt.hist(fa["NASSIGN"], bins=np.arange(0,26), histtype="step")
plt.ylabel("Count")
plt.xlabel("Number of exposures (1000s)")
# plt.yscale("log")

In [ ]:
Table.from_pandas(fa).write("LSST_WLY1_XMM_LSS_fiber_assigned.fits")